In [1]:
import numpy as np
import pandas as pd

races = [
  'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE',
  'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'H_MALE', 'H_FEMALE',
  'TOM_MALE', 'TOM_FEMALE'
]

zips = pd.read_csv('./data/county_to_zip.csv', dtype=str)
zips = zips.set_index('COUNTY')
zips = zips[['ZIP']]
zips['ZIP'] = zips['ZIP'].str.slice(stop=3)
zips = zips.drop_duplicates()

census = pd.read_csv('./data/census.csv', dtype={'STATE': str, 'COUNTY': str})
census = census[census['YEAR'] == 8]
census['COUNTY'] = census['STATE'] + census['COUNTY']

census_race = census.set_index('COUNTY')
census_race = census_race[census_race['AGEGRP'] == 0]
census_race = census_race[races].div(census_race['TOT_POP'], axis=0)

ages = {}
for i in range(1, 19): ages[i] = 'AGE_' + str(i)
    
census_age = census.set_index(['COUNTY', 'AGEGRP'])['TOT_POP'].unstack('AGEGRP')
del census_age.columns.name
census_age = census_age.iloc[:,1:].div(census_age[0], axis=0)
census_age = census_age.rename(columns=ages, index=str)

df = pd.merge(zips, census_race, left_index=True, right_index=True)
df = pd.merge(df, census_age, left_index=True, right_index=True)
df = df.reset_index().set_index('ZIP')

accepted = pd.read_csv('./data/loans.csv', header=1, dtype=str)
accepted['zip_code'] = accepted['zip_code'].str.slice(stop=3)
accepted = accepted[['zip_code', 'loan_status']]
accepted['COUNT'] = True
accepted = accepted.groupby(['zip_code', 'loan_status']).count()['COUNT'].unstack('loan_status').fillna(0)
accepted['ACCEPTED_COUNT'] = accepted.sum(axis=1)
accepted = accepted.astype(int)

statuses = {
  'Charged Off': 'STATUS_CHARGED_OFF',
  'Current': 'STATUS_CURRENT',
  'Default': 'STATUS_DEFAULT',
  'Fully Paid': 'STATUS_FULLY_PAID',
  'In Grace Period': 'STATUS_IN_GRACE_PERIOD',
  'Late (16-30 days)': 'STATUS_LATE_1',
  'Late (31-120 days)': 'STATUS_LATE_2',
}

accepted = accepted.rename(columns=statuses, index=str)
del accepted.columns.name
accepted.index.name = 'ZIP'

rejected = pd.read_csv('./data/loan_rejections.csv', header=1)
rejected = rejected['Zip Code'].str.slice(stop=3).value_counts().to_frame()
rejected = rejected.rename(columns={'Zip Code': 'REJECT_COUNT'}, index=str)
rejected.index.name = 'ZIP'

df = pd.merge(df, accepted, left_index=True, right_index=True)
df = pd.merge(df, rejected, left_index=True, right_index=True)

df = df.reset_index().sort_values(['ZIP', 'COUNTY']).reset_index(drop=True)

df.to_csv('./data/stats.csv', index=False)